In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def weighted_divisor_sum(k,r):
    ans=0
    pfk=prime_factors(k)
    weights=[]
    for d in divisors(k):
        weight=1
        pfd=prime_factors(d)
        for p in pfd:
            opd=order(p,d)
            opk=order(p,k)
            wt=binomial(opk,opd)
            weight=weight*wt
        weights=weights+[weight]
        ans=ans+weight*d^r
    return ans

def prime_factors(n):
    F=factor(n)
    f0=[f[0] for f in F]
    return sortAndSmash(f0)

def omega(n):
    if n==1:return 1
    if n>1:return len(prime_factors(n))

def omega2(n):
    return len(prime_factors(n))

def bigOmega(n):
    F=factor(n)
    ans=0
    for f in F:
        ans=ans+f[1]
    return ans

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [2]:
wtDvSrs=0
for n in [1..10]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,0)*x^n
print(wtDvSrs)
print(wtDvSrs.truncate(4))

4*x^10 + 4*x^9 + 8*x^8 + 2*x^7 + 4*x^6 + 2*x^5 + 4*x^4 + 2*x^3 + 2*x^2 + x
2*x^3 + 2*x^2 + x


In [6]:
wtDvSrs=0
for n in [1..10000]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,0)*x^n
    
import time

def recip_wtDv(n):
    g=L(1/wtDvSrs.truncate(2*n))
    return g

def recip_wtDvPowerConstant(n):
    f=recip_wtDv(n+1)^n
    return f[0]


start=time.time()

    
L=LaurentSeriesRing(QQ,'x', default_prec=11000)
import pickle
datafile= open('/Users/barrybrent/run16may23no1.txt','wb') 
data=[]
start=time.time()/3600
for a in [1..1000]:
    c=recip_wtDvPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(1, 6.219663191586733e-06)
(2, 0.00026075553614646196)
(3, 0.0008062302367761731)
(4, 0.0013163457624614239)
(5, 0.0020860129734501243)
(6, 0.0028608579887077212)
(7, 0.003911973035428673)
(8, 0.004657502693589777)
(9, 0.005679058260284364)
(10, 0.006675447977613658)
(11, 0.007937926915474236)
(12, 0.008926330192480236)
(13, 0.010194324073381722)
(14, 0.01145583763718605)
(15, 0.01299366046441719)
(16, 0.013939624652266502)
(17, 0.015163404925260693)
(18, 0.01637977024074644)
(19, 0.01785907690646127)
(20, 0.019071061571594328)
(21, 0.020562857680488378)
(22, 0.02205155190313235)
(23, 0.023809038219042122)
(24, 0.025015873834490776)
(25, 0.02650253742467612)
(26, 0.02800199156627059)
(27, 0.02977758046472445)
(28, 0.03124712937278673)
(29, 0.03300557273905724)
(30, 0.03476094244979322)
(31, 0.03678456740453839)
(32, 0.037957113003358245)
(33, 0.03945686295628548)
(34, 0.04088715632678941)
(35, 0.042604697693604976)
(36, 0.044042760506272316)
(37, 0.045750384393613786)
(38, 0.0474485868

(314, 0.7167470115819015)
(315, 0.7199047816102393)
(316, 0.7228022165945731)
(317, 0.7259826577501372)
(318, 0.7291519712889567)
(319, 0.7325734563637525)
(320, 0.7346490496420301)
(321, 0.7370049312594347)
(322, 0.739392876625061)
(323, 0.7421270461054519)
(324, 0.7444924921146594)
(325, 0.747116714133881)
(326, 0.7497521890909411)
(327, 0.7526493938057683)
(328, 0.7550078599597327)
(329, 0.7577623502002098)
(330, 0.76051022100728)
(331, 0.7635089463437907)
(332, 0.7662390160840005)
(333, 0.7692268691025674)
(334, 0.772215471311938)
(335, 0.7754828212782741)
(336, 0.7779436741839163)
(337, 0.7806618788163178)
(338, 0.7833939454867505)
(339, 0.7863811026909389)
(340, 0.7891092702047899)
(341, 0.7921326876967214)
(342, 0.7951266985619441)
(343, 0.7984089818783104)
(344, 0.8011342346435413)
(345, 0.804123203502968)
(346, 0.8071125313290395)
(347, 0.810370385996066)
(348, 0.8133613682584837)
(349, 0.8166209508199245)
(350, 0.8198702204390429)
(351, 0.823398684617132)
(352, 0.825891591084

(632, 1.7091539044049568)
(633, 1.7127369566587731)
(634, 1.716335378238)
(635, 1.7202039810363203)
(636, 1.7237841763417237)
(637, 1.7276614957954735)
(638, 1.7315099166007712)
(639, 1.7356373652000912)
(640, 1.7381479679606855)
(641, 1.740935246576555)
(642, 1.7437219871790148)
(643, 1.746753957180772)
(644, 1.7495399629697204)
(645, 1.7525903458008543)
(646, 1.7556351679959334)
(647, 1.7589853177196346)
(648, 1.7617673155036755)
(649, 1.7648211874184199)
(650, 1.7678734338842332)
(651, 1.7712108580162749)
(652, 1.7742688571452163)
(653, 1.7775858671520837)
(654, 1.7808987034950405)
(655, 1.7845067740418017)
(656, 1.7873084904858842)
(657, 1.7903589057386853)
(658, 1.7934243301860988)
(659, 1.796740313002374)
(660, 1.7997985701658763)
(661, 1.8031244007288478)
(662, 1.8064583157538436)
(663, 1.8100255877361633)
(664, 1.813079348532483)
(665, 1.8163932288298383)
(666, 1.8197145521407947)
(667, 1.8233260954730213)
(668, 1.826663805230055)
(669, 1.830263177107554)
(670, 1.83390161045826

(952, 2.8415101824793965)
(953, 2.8454709696234204)
(954, 2.8494755982537754)
(955, 2.853857918526046)
(956, 2.8578590491088107)
(957, 2.862121107405983)
(958, 2.8663461340474896)
(959, 2.8708875629818067)
(960, 2.8740477361134253)
(961, 2.8774743258254603)
(962, 2.8809059435734525)
(963, 2.88459021988092)
(964, 2.8880182313150726)
(965, 2.8917351795826107)
(966, 2.895444233843591)
(967, 2.899383334966842)
(968, 2.902892027166672)
(969, 2.90658673690632)
(970, 2.9102495019324124)
(971, 2.9142115102149546)
(972, 2.917916022473946)
(973, 2.9218809852027334)
(974, 2.9258829249301925)
(975, 2.9301135516143404)
(976, 2.933531738817692)
(977, 2.9372189368586987)
(978, 2.940923162677791)
(979, 2.9448828237946145)
(980, 2.9485560745815746)
(981, 2.952500554616563)
(982, 2.956459386856295)
(983, 2.9607105649774894)
(984, 2.964410528249573)
(985, 2.968375481548719)
(986, 2.9723319351905957)
(987, 2.976567999983672)
(988, 2.9805402974016033)
(989, 2.9847833149833605)
(990, 2.9890384243335575)
(99

In [2]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
print(s[0]);print(s[1]);
print(len(s))

(1, -2)
(2, 8)
1000


In [2]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    m2=ZZ(mod(c,2))
    data=data+[m2]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [6]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    o2a=order(2,a)
    o3a=order(3,a)
    c=pair[1]
    o2c=order(2,c)
    ds=digitsum(2,a)
    print((a,ds,o2a,o3a,o2c))

(1, 1, 0, 0, 1)
(2, 1, 1, 0, 3)
(3, 2, 0, 1, 2)
(4, 1, 2, 0, 4)
(5, 2, 0, 0, 2)
(6, 2, 1, 1, 2)
(7, 3, 0, 0, 3)
(8, 1, 3, 0, 4)
(9, 2, 0, 2, 2)
(10, 2, 1, 0, 2)
(11, 3, 0, 0, 3)
(12, 2, 2, 1, 4)
(13, 3, 0, 0, 2)
(14, 3, 1, 0, 3)
(15, 4, 0, 1, 4)
(16, 1, 4, 0, 5)
(17, 2, 0, 0, 4)
(18, 2, 1, 2, 2)
(19, 3, 0, 0, 3)
(20, 2, 2, 0, 3)
(21, 3, 0, 1, 2)
(22, 3, 1, 0, 2)
(23, 4, 0, 0, 4)
(24, 2, 3, 1, 4)
(25, 3, 0, 0, 2)
(26, 3, 1, 0, 8)
(27, 4, 0, 3, 3)
(28, 3, 2, 0, 3)
(29, 4, 0, 0, 6)
(30, 4, 1, 1, 2)
(31, 5, 0, 0, 5)
(32, 1, 5, 0, 6)
(33, 2, 0, 1, 2)
(34, 2, 1, 0, 3)
(35, 3, 0, 0, 4)
(36, 2, 2, 2, 4)
(37, 3, 0, 0, 2)
(38, 3, 1, 0, 4)
(39, 4, 0, 1, 4)
(40, 2, 3, 0, 4)
(41, 3, 0, 0, 4)
(42, 3, 1, 1, 2)
(43, 4, 0, 0, 3)
(44, 3, 2, 0, 3)
(45, 4, 0, 2, 2)
(46, 4, 1, 0, 2)
(47, 5, 0, 0, 5)
(48, 2, 4, 1, 10)
(49, 3, 0, 0, 3)
(50, 3, 1, 0, 5)
(51, 4, 0, 1, 3)
(52, 3, 2, 0, 3)
(53, 4, 0, 0, 3)
(54, 4, 1, 3, 4)
(55, 5, 0, 0, 4)
(56, 3, 3, 0, 4)
(57, 4, 0, 1, 2)
(58, 4, 1, 0, 2)
(59, 5, 0, 0, 10)
(60,

In [7]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o2=order(2,c)
    if o2==1:data=data+[a]
    print((a,o2))
print();print(data)

(1, 1)
(2, 3)
(3, 2)
(4, 4)
(5, 2)
(6, 2)
(7, 3)
(8, 4)
(9, 2)
(10, 2)
(11, 3)
(12, 4)
(13, 2)
(14, 3)
(15, 4)
(16, 5)
(17, 4)
(18, 2)
(19, 3)
(20, 3)
(21, 2)
(22, 2)
(23, 4)
(24, 4)
(25, 2)
(26, 8)
(27, 3)
(28, 3)
(29, 6)
(30, 2)
(31, 5)
(32, 6)
(33, 2)
(34, 3)
(35, 4)
(36, 4)
(37, 2)
(38, 4)
(39, 4)
(40, 4)
(41, 4)
(42, 2)
(43, 3)
(44, 3)
(45, 2)
(46, 2)
(47, 5)
(48, 10)
(49, 3)
(50, 5)
(51, 3)
(52, 3)
(53, 3)
(54, 4)
(55, 4)
(56, 4)
(57, 2)
(58, 2)
(59, 10)
(60, 5)
(61, 2)
(62, 3)
(63, 6)
(64, 11)
(65, 3)
(66, 2)
(67, 3)
(68, 5)
(69, 3)
(70, 2)
(71, 8)
(72, 5)
(73, 2)
(74, 3)
(75, 3)
(76, 5)
(77, 3)
(78, 2)
(79, 5)
(80, 5)
(81, 2)
(82, 2)
(83, 5)
(84, 3)
(85, 2)
(86, 3)
(87, 4)
(88, 4)
(89, 3)
(90, 3)
(91, 3)
(92, 3)
(93, 2)
(94, 10)
(95, 6)
(96, 6)
(97, 5)
(98, 4)
(99, 4)
(100, 3)
(101, 5)
(102, 2)
(103, 4)
(104, 4)
(105, 2)
(106, 2)
(107, 4)
(108, 3)
(109, 4)
(110, 4)
(111, 5)
(112, 5)
(113, 4)
(114, 3)
(115, 3)
(116, 3)
(117, 2)
(118, 3)
(119, 5)
(120, 4)
(121, 2)
(122, 6)
(123, 

In [8]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o2=order(2,c)
    if o2==2:data=data+[a]
    print((a,o2))
print();print(data)

(1, 1)
(2, 3)
(3, 2)
(4, 4)
(5, 2)
(6, 2)
(7, 3)
(8, 4)
(9, 2)
(10, 2)
(11, 3)
(12, 4)
(13, 2)
(14, 3)
(15, 4)
(16, 5)
(17, 4)
(18, 2)
(19, 3)
(20, 3)
(21, 2)
(22, 2)
(23, 4)
(24, 4)
(25, 2)
(26, 8)
(27, 3)
(28, 3)
(29, 6)
(30, 2)
(31, 5)
(32, 6)
(33, 2)
(34, 3)
(35, 4)
(36, 4)
(37, 2)
(38, 4)
(39, 4)
(40, 4)
(41, 4)
(42, 2)
(43, 3)
(44, 3)
(45, 2)
(46, 2)
(47, 5)
(48, 10)
(49, 3)
(50, 5)
(51, 3)
(52, 3)
(53, 3)
(54, 4)
(55, 4)
(56, 4)
(57, 2)
(58, 2)
(59, 10)
(60, 5)
(61, 2)
(62, 3)
(63, 6)
(64, 11)
(65, 3)
(66, 2)
(67, 3)
(68, 5)
(69, 3)
(70, 2)
(71, 8)
(72, 5)
(73, 2)
(74, 3)
(75, 3)
(76, 5)
(77, 3)
(78, 2)
(79, 5)
(80, 5)
(81, 2)
(82, 2)
(83, 5)
(84, 3)
(85, 2)
(86, 3)
(87, 4)
(88, 4)
(89, 3)
(90, 3)
(91, 3)
(92, 3)
(93, 2)
(94, 10)
(95, 6)
(96, 6)
(97, 5)
(98, 4)
(99, 4)
(100, 3)
(101, 5)
(102, 2)
(103, 4)
(104, 4)
(105, 2)
(106, 2)
(107, 4)
(108, 3)
(109, 4)
(110, 4)
(111, 5)
(112, 5)
(113, 4)
(114, 3)
(115, 3)
(116, 3)
(117, 2)
(118, 3)
(119, 5)
(120, 4)
(121, 2)
(122, 6)
(123, 

In [9]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o2=order(2,c)
    if o2==2:
        print(a.digits(2))

[1, 1]
[1, 0, 1]
[0, 1, 1]
[1, 0, 0, 1]
[0, 1, 0, 1]
[1, 0, 1, 1]
[0, 1, 0, 0, 1]
[1, 0, 1, 0, 1]
[0, 1, 1, 0, 1]
[1, 0, 0, 1, 1]
[0, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 0, 1]
[0, 1, 0, 1, 0, 1]
[1, 0, 1, 1, 0, 1]
[0, 1, 1, 1, 0, 1]
[1, 0, 0, 1, 1, 1]
[0, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 1]
[0, 1, 1, 0, 0, 0, 1]
[1, 0, 0, 1, 0, 0, 1]
[0, 1, 1, 1, 0, 0, 1]
[1, 0, 0, 0, 1, 0, 1]
[0, 1, 0, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 1]
[1, 0, 1, 1, 1, 0, 1]
[0, 1, 1, 0, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[0, 1, 0, 1, 0, 1, 1]
[1, 0, 1, 0, 1, 1, 1]
[1, 0, 0, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 0, 0, 0, 1]
[0, 1, 0, 1, 0, 0, 0, 1]
[1, 0, 1, 1, 0, 0, 0, 1]
[1, 0, 0, 0, 1, 0, 0, 1]
[0, 1, 1, 0, 1, 0, 0, 1]
[1, 0, 1, 1, 1, 0, 0, 1]
[0, 1, 0, 0, 0, 1, 0, 1]
[1, 0, 1, 0, 0, 1, 0, 1]
[0, 1, 1, 0, 0, 1, 0, 1]
[1, 0, 0, 0, 1, 1, 0, 1]
[0, 1, 0, 0, 1, 1, 0, 1]
[0, 1, 1, 1, 1, 1, 0, 1]
[1, 0, 0, 0, 0, 0, 1, 1]
[0, 1, 1, 0, 0, 0, 1, 1]
[1, 0, 0, 1, 0, 0, 

In [10]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    o2=order(2,c)
    if o2==2:
        print(a.digits(3))

[0, 1]
[2, 1]
[0, 2]
[0, 0, 1]
[1, 0, 1]
[1, 1, 1]
[0, 0, 2]
[0, 1, 2]
[1, 1, 2]
[1, 2, 2]
[0, 1, 0, 1]
[0, 2, 0, 1]
[1, 0, 1, 1]
[0, 2, 1, 1]
[0, 0, 2, 1]
[1, 0, 2, 1]
[0, 1, 0, 2]
[1, 1, 0, 2]
[1, 2, 0, 2]
[0, 1, 1, 2]
[1, 2, 1, 2]
[1, 0, 2, 2]
[0, 2, 2, 2]
[0, 0, 0, 0, 1]
[1, 0, 0, 0, 1]
[1, 1, 0, 0, 1]
[0, 1, 1, 0, 1]
[0, 1, 2, 0, 1]
[0, 2, 2, 0, 1]
[1, 2, 2, 0, 1]
[0, 0, 1, 1, 1]
[1, 1, 1, 1, 1]
[0, 0, 2, 1, 1]
[1, 1, 2, 1, 1]
[1, 2, 2, 1, 1]
[0, 1, 0, 2, 1]
[0, 2, 0, 2, 1]
[1, 0, 1, 2, 1]
[0, 2, 1, 2, 1]
[1, 1, 2, 2, 1]
[0, 0, 0, 0, 2]
[0, 1, 0, 0, 2]
[1, 1, 0, 0, 2]
[0, 2, 1, 0, 2]
[1, 2, 1, 0, 2]
[1, 0, 0, 1, 2]
[1, 1, 0, 1, 2]
[0, 0, 1, 1, 2]
[0, 1, 1, 1, 2]
[1, 2, 1, 1, 2]
[0, 1, 2, 1, 2]
[0, 2, 2, 1, 2]
[1, 0, 0, 2, 2]
[0, 2, 0, 2, 2]
[0, 0, 1, 2, 2]
[1, 0, 1, 2, 2]
[1, 1, 2, 2, 2]
[1, 2, 0, 0, 0, 1]
[1, 0, 1, 0, 0, 1]
[0, 0, 2, 0, 0, 1]
[1, 0, 2, 0, 0, 1]
[0, 0, 0, 1, 0, 1]
[0, 1, 0, 1, 0, 1]
[1, 2, 0, 1, 0, 1]
[0, 1, 1, 1, 0, 1]
[0, 0, 0, 2, 0, 1]
[1, 1, 0, 2, 0, 1]
[0, 0,

In [11]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    m3=ZZ(mod(c,3))
    data=data+[m3]
print(data)

[1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 2, 2, 0, 1, 2, 1, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 1, 2, 2, 0, 1, 0, 2, 2, 1, 2, 1, 1, 0, 1, 0, 1, 0, 2, 2, 2, 1, 1, 1, 2, 1, 0, 2, 2, 0, 1, 2, 1, 1, 1, 0, 2, 2, 2, 1, 0, 0, 0, 1, 2, 2, 1, 2, 2, 2, 0, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 0, 0, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 0, 2, 1, 2, 2, 0, 0, 0, 1, 0, 2, 0, 2, 0, 1, 1, 2, 0, 2, 0, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 0, 2, 1, 0, 2, 1, 0, 2, 2, 0, 1, 1, 2, 2, 0, 2, 0, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 2, 2, 2, 0, 1, 1, 0, 2, 1, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 1, 1, 0, 2, 0, 2, 0, 0, 2, 0, 0, 1, 0, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 0, 1, 0, 1, 0, 1, 2, 2, 2, 2, 1, 2, 0, 1, 1, 1, 2, 1, 0, 2, 2, 0, 1, 2, 2, 1, 2, 1, 1, 1, 0, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 0, 1, 0, 0, 2, 0, 0, 1, 0, 1, 1, 2, 1, 1, 2, 2, 1, 1, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 1, 2, 1, 1, 1, 2, 2, 2, 0, 1, 1, 0, 2, 1, 0, 2, 1, 2, 2, 1, 0, 2, 2, 2, 1, 2, 0, 1, 2, 2, 1, 1, 1, 2, 1, 

In [12]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    m3=ZZ(mod(c,3))
    if m3==0:data=data+[a]
print(data)

[10, 13, 22, 30, 32, 39, 41, 43, 52, 55, 61, 66, 67, 68, 76, 90, 91, 92, 96, 97, 98, 112, 117, 118, 119, 121, 123, 125, 129, 131, 142, 145, 148, 151, 156, 158, 163, 165, 167, 172, 175, 178, 183, 185, 190, 196, 198, 200, 201, 203, 204, 206, 226, 228, 230, 238, 244, 247, 256, 270, 272, 273, 275, 276, 278, 288, 290, 291, 293, 294, 296, 298, 307, 310, 313, 319, 325, 336, 338, 340, 343, 351, 353, 354, 356, 357, 359, 361, 363, 364, 365, 369, 371, 375, 377, 387, 388, 389, 391, 393, 395, 403, 406, 409, 412, 421, 426, 428, 435, 437, 442, 444, 446, 453, 455, 463, 466, 468, 470, 474, 475, 476, 481, 484, 489, 491, 495, 496, 497, 499, 501, 503, 514, 516, 518, 520, 525, 526, 527, 532, 534, 536, 549, 551, 555, 556, 557, 570, 572, 588, 590, 592, 594, 595, 596, 600, 601, 602, 603, 604, 605, 607, 609, 610, 611, 612, 613, 614, 618, 620, 628, 631, 643, 652, 678, 680, 684, 686, 690, 692, 714, 715, 716, 732, 734, 739, 741, 743, 745, 757, 768, 770, 775, 778, 787, 790, 793, 810, 811, 812, 816, 818, 819, 820, 

In [13]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    m3=ZZ(mod(c,3))
    if m3==1:data=data+[a]
print(data)

[1, 3, 8, 9, 14, 16, 17, 20, 23, 24, 27, 31, 35, 37, 38, 40, 42, 47, 48, 49, 51, 56, 58, 59, 60, 65, 69, 72, 77, 80, 81, 85, 86, 89, 93, 94, 101, 103, 104, 105, 106, 110, 111, 114, 120, 126, 127, 133, 134, 137, 140, 141, 144, 147, 152, 153, 160, 161, 164, 168, 173, 174, 177, 180, 188, 191, 194, 195, 205, 207, 208, 211, 212, 215, 216, 221, 223, 224, 227, 229, 231, 236, 239, 240, 241, 243, 248, 251, 253, 254, 255, 258, 259, 262, 263, 266, 267, 271, 277, 279, 280, 282, 283, 286, 287, 299, 301, 302, 303, 308, 309, 312, 315, 318, 323, 326, 329, 330, 331, 333, 334, 341, 342, 346, 347, 350, 352, 358, 360, 367, 368, 374, 378, 381, 386, 392, 398, 399, 400, 402, 407, 410, 411, 416, 418, 419, 420, 423, 427, 430, 431, 432, 436, 440, 441, 445, 449, 452, 456, 457, 459, 464, 467, 472, 473, 479, 480, 483, 488, 490, 492, 500, 502, 504, 509, 512, 515, 517, 519, 522, 523, 530, 531, 538, 539, 540, 541, 544, 545, 548, 550, 553, 554, 559, 560, 563, 564, 565, 569, 571, 573, 574, 577, 578, 581, 582, 585, 586,

In [14]:
import pickle
rfile=open('/Users/barrybrent/run16may23no1.txt','rb') 
rs=pickle.load(rfile)
rfile.close
s=stripQuotationMarks(rs)
data=[]
for k in [0..999]:
    pair=s[k]
    a=pair[0]
    c=pair[1]
    m3=ZZ(mod(c,3))
    if m3==2:data=data+[a]
print(data)

[2, 4, 5, 6, 7, 11, 12, 15, 18, 19, 21, 25, 26, 28, 29, 33, 34, 36, 44, 45, 46, 50, 53, 54, 57, 62, 63, 64, 70, 71, 73, 74, 75, 78, 79, 82, 83, 84, 87, 88, 95, 99, 100, 102, 107, 108, 109, 113, 115, 116, 122, 124, 128, 130, 132, 135, 136, 138, 139, 143, 146, 149, 150, 154, 155, 157, 159, 162, 166, 169, 170, 171, 176, 179, 181, 182, 184, 186, 187, 189, 192, 193, 197, 199, 202, 209, 210, 213, 214, 217, 218, 219, 220, 222, 225, 232, 233, 234, 235, 237, 242, 245, 246, 249, 250, 252, 257, 260, 261, 264, 265, 268, 269, 274, 281, 284, 285, 289, 292, 295, 297, 300, 304, 305, 306, 311, 314, 316, 317, 320, 321, 322, 324, 327, 328, 332, 335, 337, 339, 344, 345, 348, 349, 355, 362, 366, 370, 372, 373, 376, 379, 380, 382, 383, 384, 385, 390, 394, 396, 397, 401, 404, 405, 408, 413, 414, 415, 417, 422, 424, 425, 429, 433, 434, 438, 439, 443, 447, 448, 450, 451, 454, 458, 460, 461, 462, 465, 469, 471, 477, 478, 482, 485, 486, 487, 493, 494, 498, 505, 506, 507, 508, 510, 511, 513, 521, 524, 528, 529, 5